In [102]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
#from bs4 import BeautifulSoup
import time
import pandas as pd

In [104]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
CHROMEPATH = r'C:path'

In [58]:

login_xpath = r'/html/body/div[2]/div/div/div/div/div/form/input[3]'
url = 'https://skrapp.io/login'
link_url = 'https://www.linkedin.com/'
CHROMEPATH = r'C:\path\chromedriver.exe'
#'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe'

# Calling the chrome browser


browser = webdriver.Chrome(CHROMEPATH, options = chrome_options)
browser.set_window_position(0,0)


{'height': 708, 'width': 1050, 'x': 0, 'y': 0}

In [105]:
def create_incognito_browsers(n):
    browsers = []
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    
    for i in range(n):
        browser = webdriver.Chrome(CHROMEPATH,options = chrome_options)
        browser.set_window_position(0,0)

        browser.get('https://www.google.com')
        browser.maximize_window()

        browsers.append(browser)
       
    
    return browsers

In [106]:
def segment(n,i,urls):
    return urls[n*(i-1):n*i]

In [124]:

def get_top_details(browser):
    soupp = BeautifulSoup(browser.page_source)
    a = soupp.find('main',attrs = {'role':'main','class':'main'}).find('section',attrs = {'class':'core-rail'},recursive = False)
    b = a.find('section',attrs= {'top-card-layout'}).find('div',attrs= {'top-card-layout__card'}).find_all('div',recursive = False)
    if len(b)>0:
        follower = b[0].find('h3').text.strip()
        follower = follower.split()[-2]
        follower = int(''.join(follower.split(',') ))
        company_name =  b[0].find('h1').text.strip()
        company_motto =  b[0].find('h2').text.strip()
        c = b[0].find_all('div',recursive = False)
        if len(c)>0:
            employees = int(c[-1].find('a').text.strip().split()[-2])
        else :
            employees = None

        return [follower,company_motto,employees]
    return [None,None,None]       


In [125]:
headers = ['website','industries','company size','headquarters','type','founded','specialties']
def get_data(browser,headers = headers):
    soupp = BeautifulSoup(browser.page_source)
    a = soupp.find('main',attrs = {'role':'main','class':'main'}).find('section',attrs = {'class':'core-rail'},recursive = False)
    b = a.find('section',attrs= {'class':'about-us section-container'})
    about_us = b.find('p',attrs= {'class':'about-us__description'}).text.strip()
    c = b.find('dl',attrs= {'class':'about-us__basic-info-list'})
    ple = c.find_all('div')
    headers = ['website','industries','company size','headquarters','type','founded','specialties']
    sdict = {a:None for a in headers}
    sh = []
    sb = []
    for i in ple:
        sdict[i.find('dt').text.strip().lower()] = i.find('dd').text.strip()

    #     sh.append(i.find('dt').text.strip().lower() )
    #     sb.append(i.find('dd').text.strip() )

    # for i in range(len(sh)):
    #     sdict[sh[i]] = sb[i]
    anns = []
    for i in headers:
        anns.append(sdict[i])
    anns.append(about_us)
    return anns

In [126]:
def get_info(browser):
    d1 = get_top_details(browser)
    d2 = get_data(browser,headers = headers)
    d1.extend(d2)
    return d1

In [127]:
def run_parralell(browsers,list_urls):
    ams = []

    
    for i in list(zip(browsers,list_urls)):
        browser,link = i
        browser.get(link)
        time.sleep(1)
        browser.execute_script("document.body.style.zoom= '30%'")
    time.sleep(15)  ## time for each browser to load page
   
    for i in list(zip(browsers,list_urls)) :
        browser,link = i
        dat = get_info(browser)
        [link].extend(dat)
        ams.append(dat)
    return ams

In [134]:
def create_incognito_browsers(n):
    browsers = []
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    
    for i in range(n):
        browser = webdriver.Chrome(CHROMEPATH,options = chrome_options)
        browser.set_window_position(0,0)

        browser.get('https://www.google.com')
        browser.maximize_window()

        browsers.append(browser)
       
    
    return browsers


def segment(n,i,urls):
    return urls[n*(i-1):n*i]


def get_top_details(browser):
    soupp = BeautifulSoup(browser.page_source)
    a = soupp.find('main',attrs = {'role':'main','class':'main'}).find('section',attrs = {'class':'core-rail'},recursive = False)
    b = a.find('section',attrs= {'top-card-layout'}).find('div',attrs= {'top-card-layout__card'}).find_all('div',recursive = False)
    if len(b)>0:
        try:
            follower = b[0].find('h3').text.strip()
            follower = follower.split()[-2]
            follower = int(''.join(follower.split(',') ))
        except :
            follower = None
        company_name =  b[0].find('h1').text.strip()
        company_motto =  b[0].find('h2').text.strip()
        c = b[0].find_all('div',recursive = False)
        if len(c)>0:
            employees = int(c[-1].find('a').text.strip().split()[-2])
        else :
            employees = None

        return [follower,company_motto,employees]
    return [None,None,None]       


headers = ['website','industries','company size','headquarters','type','founded','specialties']
def get_data(browser,headers = headers):
    soupp = BeautifulSoup(browser.page_source)
    a = soupp.find('main',attrs = {'role':'main','class':'main'}).find('section',attrs = {'class':'core-rail'},recursive = False)
    b = a.find('section',attrs= {'class':'about-us section-container'})
    about_us = b.find('p',attrs= {'class':'about-us__description'}).text.strip()
    c = b.find('dl',attrs= {'class':'about-us__basic-info-list'})
    ple = c.find_all('div')
    headers = ['website','industries','company size','headquarters','type','founded','specialties']
    sdict = {a:None for a in headers}
    sh = []
    sb = []
    for i in ple:
        sdict[i.find('dt').text.strip().lower()] = i.find('dd').text.strip()

    #     sh.append(i.find('dt').text.strip().lower() )
    #     sb.append(i.find('dd').text.strip() )

    # for i in range(len(sh)):
    #     sdict[sh[i]] = sb[i]
    anns = []
    for i in headers:
        anns.append(sdict[i])
    anns.append(about_us)
    return anns


def get_info(browser):
    d1 = get_top_details(browser)
    d2 = get_data(browser,headers = headers)
    d1.extend(d2)
    return d1


def run_parralell(browsers,list_urls):
    ams = []
    for i in list(zip(browsers,list_urls)):
        browser,link = i
        browser.get(link)
        time.sleep(1)
        browser.execute_script("document.body.style.zoom= '30%'")
    time.sleep(15)  ## time for each browser to load page
   
    for i in list(zip(browsers,list_urls)) :
        browser,link = i
        dat = get_info(browser)
        [link].extend(dat)
        ams.append(dat)
    return ams


# Load Data Here

In [113]:
df = pd.read_excel('elite.xlsx',sheet_name = 'Sheet2')
urls = df['company_link']
urls

0      https://www.linkedin.com/company/wolf-&-shepherd/
1         https://www.linkedin.com/company/story-health/
2      https://www.linkedin.com/company/element-apothec/
3          https://www.linkedin.com/company/secureframe/
4       https://www.linkedin.com/company/encantosbrands/
                             ...                        
550           https://www.linkedin.com/company/steeleye/
551                                                  NaN
552                                                  NaN
553           https://www.linkedin.com/company/get-chip/
554                                                  NaN
Name: company_link, Length: 555, dtype: object

# Work Here

In [114]:
n = int(input('enter no of browser').strip())

enter no of browser5


In [117]:
browsers = create_incognito_browsers(n)

In [140]:
n=2

In [141]:
ans = []
i = 1

In [142]:
n=1

In [143]:
i

1

In [144]:
j=i
for i in range(i,int(len(urls)/n +1)) : #len(urls)
    lst = run_parralell(browsers,segment(n,i,urls))
    ans.extend(lst)
    print(i,lst,'is done')

1 [[2434, 'Apparel & Fashion', 22, 'http://www.wolfandshepherd.com', 'Apparel & Fashion', '11-50 employees', 'El Segundo, California', 'Privately Held', '2015', 'Footwear, E-commerce, Retail, and fashion', "Wolf & Shepherd is a Los Angeles based men's fashion brand built on the principles of innovation, performance and tradition. With running shoe technology and premium Italian leathers, our shoes are designed to aide modern workers as they move throughout the day, every day."]] is done
2 [[None, 'Hospital & Health Care', 9, 'https://www.storyhealth.ai', 'Hospital & Health Care', '1-10 employees', None, 'Privately Held', '2020', None, 'Empowering Specialists with Adaptive Virtualized Care\n\nStory Health makes it quick and easy for specialists to understand medical situations and make clinical decisions that are implemented at scale through virtual protocols. The end result is more patients on optimal care sooner.']] is done
3 [[437, 'Health, Wellness and Fitness', 5, 'http://www.eleme

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
ans.extend( run_parralell(browsers,urls[n*int(len(urls)/n):]) )

In [ ]:
ans_df = pd.DataFrame(ans)
ans_df.columns = ['Company linkedin Id','company_follower','company_motto','employees_on_linkedin','website','industries','company size','headquarters','type','founded','specialties','About us']

ans_df

In [74]:

browser3 = webdriver.Chrome(CHROMEPATH,options = chrome_options)
browser.set_window_position(0,0)

{'height': 708, 'width': 1050, 'x': 0, 'y': 0}

In [79]:

def get_top_details(browser):
    soupp = BeautifulSoup(browser.page_source)
    a = soupp.find('main',attrs = {'role':'main','class':'main'}).find('section',attrs = {'class':'core-rail'},recursive = False)
    b = a.find('section',attrs= {'top-card-layout'}).find('div',attrs= {'top-card-layout__card'}).find_all('div',recursive = False)
    if len(b)>0:
        follower = b[0].find('h3').text.strip()
        follower = follower.split()[-2]
        follower = int(''.join(follower.split(',') ))
        company_name =  b[0].find('h1').text.strip()
        company_motto =  b[0].find('h2').text.strip()
        c = b[0].find_all('div',recursive = False)
        if len(c)>0:
            employees = int(c[-1].find('a').text.strip().split()[-2])
        else :
            employees = None

        return [follower,company_motto,employees]
    return [None,None,None]       


In [80]:
browser3.execute_script("document.body.style.zoom = '70%'")
get_top_details(browser3)

[373, 'Computer Software', 9]

In [95]:
get_data(browser3)

['http://cosell.io',
 'Computer Software',
 '1-10 employees',
 'Austin, Texas',
 'Privately Held',
 '2020',
 None,
 'Unlock the full potential of your most valuable partnerships by selling as an aligned team in real-time. Massively expand your connected customer base by exchanging powerful introductions with your partners into key accounts. CoSell is a secure partnership collaboration network that reveals your most strategic partners with account overlaps for revenue growth.\n\nScale your revenue by establishing trusted relationships with your strategic partners. Embrace your competitive advantage with the best warm introductions from the right strategic partners. Manage all of your partner relationships and communication securely from one powerful, sleek and unified platform.']

In [89]:
anns

['http://cosell.io',
 'Computer Software',
 '1-10 employees',
 'Austin, Texas',
 'Privately Held',
 '2020',
 None]

In [69]:
browser.get('https://www.linkedin.com/in/brendon/')

In [43]:
sopp = BeautifulSoup(browser.page_source)

In [96]:
def _get_company_linkedin(a):
    #a = sopp.find('main',attrs = {'class':'core-rail'})
    g = a.find('section', attrs = {'id':'experience-section'}).find('ul')
    h =g.find_all('li')
    Company_mail = 'https://www.linkedin.com'+h[0].find('a')['href']
    return Company_mail
    

In [46]:
g = a.find('section', attrs = {'id':'experience-section'}).find('ul')

In [47]:
h =g.find_all('li')

In [71]:
Company_mail = 'https://www.linkedin.com'+h[0].find('a')['href']

In [72]:
Company_mail

'https://www.linkedin.com/company/cosell/'

In [75]:
browser3.get(Company_mail)

In [15]:
b = a.find('section',attrs= {'top-card-layout'}).find('div',attrs= {'top-card-layout__card'}).find_all('div',recursive = False)

In [16]:
len(b)

2

In [22]:
follower = b[0].find('h3').text.strip()

In [26]:
follower.split(' ')[-2]

'1,901'

In [27]:
company_name =  b[0].find('h1').text.strip()

In [28]:
company_motto =  b[0].find('h2').text.strip()
company_motto

'Information Technology and Services'

In [36]:
c = b[0].find_all('div',recursive = False)

In [37]:
len(c)

2

In [41]:
employees = int(c[-1].find('a').text.strip().split()[-2])
employees

8